## Sequence to Sequence modelling

simple implementation of sequence to sequence modelling in keras. The task is to translate short English sentences into French sentences, character-by-character using a sequence-to-sequence model.

### summary of our implementation:

- We start with input sequences from a domain (e.g. English sentences)and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context. It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]` given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1 (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character
    - Sample the next character using these predictions (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we hit the character limit.
    
### Download the data

http://www.manythings.org/anki/fra-eng.zip

### Steps:

1) Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:

- encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences
- decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containing a one-hot vectorization of the French sentences
- decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

2) Train a basic LSTM-based Seq2Seq model to predict decoder_target_data given encoder_input_data and decoder_input_data.

3) Decode some sentences to check that the model is working (i.e. turn samples from encoder_input_data into corresponding samples from decoder_target_data).



In [1]:
# import modules
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.layers import Input, LSTM, Dense

import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
os.getcwd()

'/content'

In [5]:
os.chdir('/content/drive/MyDrive/MachineTransalation')

In [6]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to traina for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "DATASET.txt"

In [7]:
# Reading the data (import data)
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

In [8]:
lines[:10]

['Wow!\tवाह!',
 'Help!\tबचाओ!',
 'Jump.\tउछलो.',
 'Jump.\tकूदो.',
 'Jump.\tछलांग.',
 'Hello!\tनमस्ते।',
 'Hello!\tनमस्कार।',
 'Cheers!\tवाह-वाह!',
 'Cheers!\tचियर्स!',
 'Got it?\tसमझे कि नहीं?']

In [9]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [10]:
#input_text, target_text, _  = lines[0].split('\t')

In [12]:
input_text[:20]

NameError: ignored

In [ ]:
#target_text

In [13]:
lines[185583]

IndexError: ignored

In [14]:
len(lines)-1

2867

In [15]:
min(num_samples, len(lines) - 1)

2867

In [16]:
#"\t" + target_text + "\n"

In [17]:
#Creating input_characters & Output Characters arrays
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [18]:
input_texts[:10]

['Wow!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Got it?']

In [19]:
target_texts[:10]

['\tवाह!\n',
 '\tबचाओ!\n',
 '\tउछलो.\n',
 '\tकूदो.\n',
 '\tछलांग.\n',
 '\tनमस्ते।\n',
 '\tनमस्कार।\n',
 '\tवाह-वाह!\n',
 '\tचियर्स!\n',
 '\tसमझे कि नहीं?\n']

In [20]:
#

In [21]:
#target_characters

In [22]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [23]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 2867
Number of unique input tokens: 70
Number of unique output tokens: 92
Max sequence length for inputs: 124
Max sequence length for outputs: 123


In [24]:
#input_characters

In [25]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [26]:
np.zeros((2,3,2))

array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])

In [27]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

In [28]:
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [29]:
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [30]:
#input_texts

In [31]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
        encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

## Build the model

In [32]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [34]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

es = EarlyStopping(monitor='val_loss',mode='min',verbose=1, patience=200)
mc = ModelCheckpoint('hinditoenglishmodel.h5',monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)

In [35]:
## Train the model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
history =model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=1000,
    validation_split=0.2,
    callbacks=[es,mc])
# Save model


Epoch 1/1000
35/36 [============================>.] - ETA: 0s - loss: 0.4956 - accuracy: 0.8685
Epoch 1: val_accuracy improved from -inf to 0.76815, saving model to hinditoenglishmodel.h5
36/36 [==============================] - 5s 70ms/step - loss: 0.4956 - accuracy: 0.8685 - val_loss: 0.8835 - val_accuracy: 0.7682
Epoch 2/1000
34/36 [===========================>..] - ETA: 0s - loss: 0.4804 - accuracy: 0.8717
Epoch 2: val_accuracy improved from 0.76815 to 0.77156, saving model to hinditoenglishmodel.h5
36/36 [==============================] - 1s 30ms/step - loss: 0.4802 - accuracy: 0.8717 - val_loss: 0.8538 - val_accuracy: 0.7716
Epoch 3/1000
34/36 [===========================>..] - ETA: 0s - loss: 0.4716 - accuracy: 0.8733
Epoch 3: val_accuracy improved from 0.77156 to 0.77297, saving model to hinditoenglishmodel.h5
36/36 [==============================] - 1s 30ms/step - loss: 0.4702 - accuracy: 0.8736 - val_loss: 0.8478 - val_accuracy: 0.7730
Epoch 4/1000
35/36 [====================

In [ ]:
os.getcwd()

In [36]:
model.save("englishtohindi",save_format='h5')

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [32]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("englishtohindi")

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 70)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 92)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        334848      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
model.input?

In [37]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="inpu t_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [38]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [39]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [40]:
seq_index=54

input_seq = encoder_input_data[seq_index : seq_index + 1]
decoded_sentence = decode_sequence(input_seq)
print("-")
print("Input sentence:", input_texts[seq_index])
print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 20ms/step
-
Input sentence: I love you.
Decoded sentence: मैं तुम्हारे जितना लम्बा हूँ।



In [41]:
#You can now generate decoded sentences as such:
for seq_index in range(5):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Wow!
Decoded sentence: मुझे अपना पता बतादेना।

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Help!
Decoded sentence: यह मुफ़्त का है।

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Jump.
Decoded sentence: मुझे अपना पता बतादेना।

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Jump.
Decoded sentence: मुझे अपना पता बतादेना।

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Jump.
Decoded sentence: मुझे अपना पता बतादेना।

